In [1]:
%run ../Python_files/extract_tmc_links_journal.py

%run ../Python_files/load_dicts.py

%run ../Python_files/util.py

import csv

In [2]:
tmc_set_link = {}
AM_capac_link = {}
MD_capac_link = {}
PM_capac_link = {}
NT_capac_link = {}

num_of_links = 258
for idx in range(num_of_links):
    idx_ = idx / 2
    if (idx % 2 == 0):
        tmc_set_link[idx] = [i for i in tmc_list_journal_link_dict[str(idx_+1)] if ('P' in i or '+' in i)]
    else:
        tmc_set_link[idx] = [i for i in tmc_list_journal_link_dict[str(idx_+1)] if ('N' in i or '-' in i)]
        
    AM_capac_link[idx] = sum([tmc_capac_dict_AM_journal[i] * tmc_length_dict_journal[i] / \
                              (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]]) / \
    sum([tmc_length_dict_journal[i] / (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]])
    
    MD_capac_link[idx] = sum([tmc_capac_dict_MD_journal[i] * tmc_length_dict_journal[i] / \
                              (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]]) / \
    sum([tmc_length_dict_journal[i] / (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]])
    
    PM_capac_link[idx] = sum([tmc_capac_dict_PM_journal[i] * tmc_length_dict_journal[i] / \
                              (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]]) / \
    sum([tmc_length_dict_journal[i] / (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]])
    
    NT_capac_link[idx] = sum([tmc_capac_dict_NT_journal[i] * tmc_length_dict_journal[i] / \
                              (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]]) / \
    sum([tmc_length_dict_journal[i] / (tmc_ref_speed_dict_journal[i]) for i in tmc_set_link[idx]])

In [3]:
ini_node_dict = {}
ter_node_dict = {}

input_file = '../00_subnetwork_topology/MA_journal_topology.csv'

i = 0
with open(input_file, 'rb') as inp:
    for row in csv.reader(inp):
        if 'origin' not in row:
            ini_node_dict[i] = int(row[0])
            ter_node_dict[i] = int(row[1])
            i += 1

In [4]:
# instantiating Link class
link = {}

for idx in range(num_of_links):
    link[idx] = Link_with_Free_Flow_Time_Journal(ini_node_dict[idx], ter_node_dict[idx], tmc_set_link[idx], \
                                                 AM_capac_link[idx], MD_capac_link[idx], \
                                                 PM_capac_link[idx], NT_capac_link[idx], 0, 0, 0, 0, 0, 0)

In [5]:
zdump([link[idx] for idx in range(num_of_links)], '../temp_files/links_with_capac_journal.pkz')

In [6]:
link_with_capac_list = list(zload('../temp_files/links_with_capac_journal.pkz'))

link_length_dict = {}

i = 0
for link_with_capac in link_with_capac_list:
    link_with_capac_new = Link_with_Free_Flow_Time_Journal(link_with_capac.init_node, link_with_capac.term_node, \
                                                   link_with_capac.tmc_set, \
                                                   link_with_capac.AM_capac, \
                                                   link_with_capac.MD_capac, \
                                                   link_with_capac.PM_capac, \
                                                   link_with_capac.NT_capac, \
                                                   link_with_capac.length, \
                                                   link_with_capac.free_flow_time, \
                                                   0, 0, 0, 0)
    link_length_dict[str(i)] = link_with_capac_new
    i = i + 1

zdump(link_length_dict, '../temp_files/link_length_dict_journal.pkz')